Fonte:

http://www.bmfbovespa.com.br/shared/iframe.aspx?idioma=pt-br&url=http://www.bmfbovespa.com.br/pt-br/cotacoes-historicas/FormSeriesHistoricas.asp

Formato:

http://www.bmfbovespa.com.br/pt-br/download/SeriesHistoricas_Layout.pdf

Descrição:

Os dados são referentes ao mês de Novembro/2015

In [43]:
using DataFrames

function parseQuotation(df, n, line)
    date = string(line[3:6],"-",line[7:8],"-",line[9:10])
    codbdi = line[11:12]
    codbdi == "02" || return # Lote Padrão

    codneg = strip(line[13:24]) # CÓDIGO DE NEGOCIAÇÃO DO PAPEL
    tpmerc = line[25:27] # CÓD. DO MERCADO EM QUE O PAPEL ESTÁ CADASTRADO
    nomres = strip(line[28:39]) # NOME RESUMIDO DA EMPRESA EMISSORA DO PAPEL
    modref = strip(line[53:56]) # MOEDA DE REFERÊNCIA
    preabe = parse(Float64, line[57:69]) / 100 # PREÇO DE ABERTURA DO PAPEL-MERCADO NO PREGÃO V(99)
    preult = parse(Float64, line[109:121]) / 100 # PREÇO DO ÚLTIMO NEGÓCIO DO PAPEL-MERCADO NO PREGÃO V(99)
    totneg = parse(Int, line[148:152]) # NÚMERO DE NEGÓCIOS EFETUADOS COM O PAPELMERCADO NO PREGÃO
    quatot = parse(Int, line[153:170]) # QUANTIDADE TOTAL DE TÍTULOS NEGOCIADOS NESTE PAPELMERCADO
    voltot = parse(Float64, line[171:188]) / 100 # VOLUME TOTAL DE TÍTULOS NEGOCIADOS NESTE PAPELMERCADO V(99)
    push!(df, [date, codbdi, codneg, tpmerc, nomres, modref, preabe, preult, totneg, quatot, voltot])
end

function parseRegistry(df, n, line)
    rtype = line[1:2] 
    if rtype == "00"
        # ignoring header
    elseif rtype == "01"
        parseQuotation(df, n, line)
    elseif rtype == "99"
        # ignoring trailer
    else
        error("($n) unknown type: $rtype")
    end
end

function loadData(filename)
    df = DataFrame(
            DATE = AbstractString[],
            CODBDI = AbstractString[],
            CODNEG = AbstractString[],
            TPMERC = AbstractString[],
            NOMRES = AbstractString[],
            MODREF = AbstractString[],
            PREABE = Float64[],
            PREULT = Float64[],
            TOTNEG = Int[],
            QUATOT = Int[],
            VOLTOT = Float64[])
    open(filename) do f
        for (n, line) in enumerate(eachline(f))
            parseRegistry(df, n, line)
        end
    end
    df
end

data = loadData("COTAHIST_M112015.TXT")

describe(data)

DATE


In [44]:
println("Total de negociações: ", round(sum(data[:TOTNEG]) / 10^6, 1))
println("Total de títulos: ", round(sum(data[:QUATOT]) / 10^6, 1))
println("Total em volume: ", round(sum(data[:VOLTOT]) / 10^6, 1))

In [45]:
k = []
n = length(unique(data[:DATE]))
for q in unique(data[:CODNEG])
    nq = nrow(data[data[:CODNEG] .== q, [:DATE]])
    nq == n || push!(k, q)
end
length(k)

242

Length  5995
Type    AbstractString
NAs     0
NA%     0.0%
Unique  19

CODBDI
Length  5995
Type    AbstractString
NAs     0
NA%     0.0%
Unique  1

CODNEG
Length  5995
Type    AbstractString
NAs     0
NA%     0.0%
Unique  451

TPMERC
Length  5995
Type    AbstractString
NAs     0
NA%     0.0%
Unique  1

NOMRES
Length  5995
Type    AbstractString
NAs     0
NA%     0.0%
Unique  369

MODREF
Length  5995
Type    AbstractString
NAs     0
NA%     0.0%
Unique  1

PREABE
Min      0.01
1st Qu.  3.15
Median   10.25
Mean     52.87168306922435
3rd Qu.  28.035
Max      47645.0
NAs      0
NA%      0.0%

PREULT
Min      0.01
1st Qu.  3.15
Median   10.28
Mean     52.90588323603003
3rd Qu.  28.045
Max      47645.0
NAs      0
NA%      0.0%

TOTNEG
Min      1.0
1st Qu.  4.0
Median   99.0
Mean     2795.8902418682237
3rd Qu.  2487.0
Max      61557.0
NAs      0
NA%      0.0%

QUATOT
Min      1.0
1st Qu.  2600.0
Median   59700.0
Mean     1.3840572552126772e6
3rd Qu.  835150.0
Max      9.26903e7
NAs      0
NA%

In [46]:
alldates = DataArray{Bool,1}(map(x -> !(x in k), data[:CODNEG]))
data = data[alldates, :]

,DATE,CODBDI,CODNEG,TPMERC,NOMRES,MODREF,PREABE,PREULT,TOTNEG,QUATOT,VOLTOT
1,2015-11-03,02,ABCB4,010,ABC BRASIL,R$,8.49,8.8,1068,257000,2.243335e6
2,2015-11-03,02,ABEV3,010,AMBEV S/A,R$,19.1,19.6,32307,13504000,2.61856419e8
3,2015-11-03,02,AGRO3,010,BRASILAGRO,R$,10.7,10.9,77,21000,226828.0
4,2015-11-03,02,ALPA3,010,ALPARGATAS,R$,9.58,9.71,14,11200,108100.0
5,2015-11-03,02,ALPA4,010,ALPARGATAS,R$,8.19,8.6,1394,718900,6.141464e6
6,2015-11-03,02,ALSC3,010,ALIANSCE,R$,10.44,11.12,1360,378700,4.191205e6
7,2015-11-03,02,ALUP11,010,ALUPAR,R$,15.7,15.35,1766,1195900,1.8348705e7
8,2015-11-03,02,AMAR3,010,LOJAS MARISA,R$,6.52,6.7,51,20600,136972.0
9,2015-11-03,02,ANIM3,010,ANIMA,R$,13.41,13.62,4261,826500,1.1304982e7
10,2015-11-03,02,ARTR3,010,ARTERIS,R$,9.37,9.37,2398,513500,4.802094e6


In [47]:
data[:RET] = (data[:PREULT] - data[:PREABE]) ./ data[:PREABE]
ret = data[:, [:CODNEG, :DATE, :RET]]
sort!(ret, cols = (:CODNEG, :DATE))
ret

,CODNEG,DATE,RET
1,ABCB4,2015-11-03,0.03651354534746767
2,ABCB4,2015-11-04,0.02156640181611799
3,ABCB4,2015-11-05,0.015538290788013382
4,ABCB4,2015-11-06,-0.016375545851528422
5,ABCB4,2015-11-09,0.006651884700665244
6,ABCB4,2015-11-10,-0.0022050716648290597
7,ABCB4,2015-11-11,0.04070407040704081
8,ABCB4,2015-11-12,-0.0010570824524314547
9,ABCB4,2015-11-13,-0.020212765957446754
10,ABCB4,2015-11-16,0.0032362459546926795


In [48]:
tidx = unique(ret[:DATE])
nidx = unique(ret[:CODNEG])
result = zeros(length(tidx), length(nidx))

for i=1:nrow(ret)
    n, t, r = ret[:CODNEG][i], ret[:DATE][i], ret[:RET][i]
    ni = findfirst(nidx, n)
    ti = findfirst(tidx, t)
    result[ti, ni] = r
end

result

19x209 Array{Float64,2}:
  0.0365135    0.026178      0.0186916    …  -0.00758621  -0.0398551 
  0.0215664   -0.00457085    0.0256645       -0.00138985   0.0       
  0.0155383   -0.00153925    0.013369         0.0694927    0.0150943 
 -0.0163755   -0.0206505     0.0166667       -0.0208062    0.0520446 
  0.00665188  -0.014233     -0.0025729       -0.00995355  -0.0178571 
 -0.00220507   0.00430802   -0.0170213    …  -0.0308104   -0.0107914 
  0.0407041    0.0187066    -0.0155844        0.0055325   -0.0150943 
 -0.00105708  -0.00839895    0.00350877      -0.0226337    0.0344828 
 -0.0202128    0.011117      0.000869565     -0.0105634    0.00371747
  0.00323625  -0.000524659  -0.0087184        0.00923952  -0.0151515 
  0.00746269  -0.00932642   -0.00615655   …   0.00980392   0.0       
  0.00844773   0.00261506    0.0             -0.0228216   -0.0194553 
  0.0366492   -0.00309119   -0.000884956      0.0352113    0.03125   
 -0.010101    -0.0154959    -0.0140474       -0.00608519  -0.0038

In [49]:
C, R = cov(result), mean(result, 1)

(
209x209 Array{Float64,2}:
  0.000533397   9.76987e-5    6.29312e-5   …   0.000157678  -6.35954e-5 
  9.76987e-5    0.000147318  -7.98739e-6      -4.18321e-5   -7.1184e-5  
  6.29312e-5   -7.98739e-6    0.000142261      3.7783e-5     3.44445e-5 
  2.1333e-5     6.43695e-5    7.79469e-6      -0.000105535  -7.90443e-5 
  0.000222631   0.000190124   0.000126063     -0.000282599  -0.00026856 
  0.000355128   0.000152915   7.17367e-5   …   9.86364e-5   -0.000173753
  0.000138977   2.98108e-5    9.70375e-6       5.94375e-5    0.000258   
  0.000421464   8.3647e-5     8.03635e-5       0.000168933  -0.000137753
  0.000269137   4.20291e-5   -3.50408e-5      -7.83959e-5   -0.000253977
 -1.53107e-5    1.25307e-6   -1.14821e-5       1.7238e-5    -2.59918e-5 
  0.000214253   3.431e-5     -5.44429e-5   …   0.000101712  -4.86168e-5 
 -0.000182446  -5.67022e-5   -5.41801e-5      -0.000198155  -2.47844e-5 
  0.000528814   0.000154016   8.60238e-5       0.000260174  -4.21134e-5 
  ⋮                    

In [54]:
#writedlm("n.tsv", nidx)
#writedlm("C.tsv", C)
#writedlm("R.tsv", R)
writetable("data.tsv", ret)

In [51]:
function save(df, name)
    tidx = unique(df[:DATE])
    nidx = unique(df[:CODNEG])
    data = zeros(length(tidx), length(nidx))

    for i=1:nrow(df)
        n, t, r = df[:CODNEG][i], df[:DATE][i], df[:RET][i]
        ni = findfirst(nidx, n)
        ti = findfirst(tidx, t)
        data[ti, ni] = r
    end

    C, R = cov(data), mean(data, 1)
    
    writedlm("$name-n.tsv", nidx)
    writedlm("$name-C.tsv", C)
    writedlm("$name-R.tsv", R)

    writetable("$name.tsv", df)
    df
end

save (generic function with 1 method)

In [52]:
# Maior rendimento médio

rdf = DataFrame(CODNEG = AbstractString[], RET = Float64[])

for q in unique(ret[:CODNEG])
    qdf = ret[ret[:CODNEG] .== q, :]
    m = mean(qdf[:RET])
    push!(rdf, [q, m])
end
sort!(rdf, cols = :RET, rev = true)

rdf

,CODNEG,RET
1,VIVR3,0.08333333333333337
2,LOGN3,0.02372840691628895
3,BRIN3,0.013053055132483858
4,PNVL3,0.012515252161818001
5,ELET3,0.01170931906051046
6,ROMI3,0.011098163067977916
7,CSMG3,0.010918432395899848
8,MAGG3,0.010895996068574212
9,BRKM5,0.008825894566491386
10,POSI3,0.008409644158418333


In [53]:
top10 = rdf[1:10,:][:CODNEG]
filterTop10 = DataArray{Bool, 1}(map(x -> x in top10, ret[:CODNEG]))
maxreturndf = ret[filterTop10, :]
save(maxreturndf, "maxreturn")

,CODNEG,DATE,RET
1,BRIN3,2015-11-03,0.036144578313253045
2,BRIN3,2015-11-04,-0.011363636363636374
3,BRIN3,2015-11-05,0.0
4,BRIN3,2015-11-06,-0.011363636363636374
5,BRIN3,2015-11-09,0.05882352941176476
6,BRIN3,2015-11-10,-0.01111111111111112
7,BRIN3,2015-11-11,0.02222222222222224
8,BRIN3,2015-11-12,0.0
9,BRIN3,2015-11-13,0.054347826086956444
10,BRIN3,2015-11-16,-0.031249999999999913
